# Functionality of this notebook: 

* calculate the ciCOH for normal and mild LFP data in rest

## Import Packages

In [1]:
import os, sys
import scipy.io as sio
import glob
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import math
import pandas as pd
%matplotlib inline

In [2]:
# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# import util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import ciCoherence_overtime in connAnalyTool/synchronization_indices.ipynb
from connAnalyTool.synchronization_indices import ciCoherence_acrosstrials
from connAnalyTool.synchronization_indices import ciCoherence_overtime

importing Jupyter notebook from /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/synchronization_indices.ipynb


## exp subfolders & code_corresfolder

In [3]:
_, _, pipelinefolder, _= exp_subfolders()

In [4]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [5]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder, correparentfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [6]:
freq = [16, 18]

halfs = False

sameDates4SKTRest = False

## input setup

In [7]:
inputfolder = os.path.join(pipelinefolder, 'NHP_Jo', '0_dataPrep', 'Rest', 
                           'm4_restData_filtered' + str(freq[0]) + '_' + str(freq[1]) + '_segDownsample')

In [8]:
variablesinLoadfile = ['lfpsegs_m1', 'lfpsegs_stn', 'lfpsegs_gp', 'fs', 'segsRemain']

In [9]:
chnInf_folder = correparentfolder
chnInf_file = os.path.join(chnInf_folder, 'chn_brainArea_simCoord_M1DBS.csv')

In [10]:
if sameDates4SKTRest:
    
    sameDatesInfFile = os.path.join(pipelinefolder, 'NHP_Jo', '0_dataPrep', 'Jo_sameDatesUsedforSTKRest.csv')

## Save Setup

In [11]:
savefolder = corresfolder
savefilename =  'ciCOH_rest' + '_freq' + str(freq[0]) + '_' + str(freq[1])

if sameDates4SKTRest:
    savefilename = savefilename + '_samedays'

## extract lfp

In [12]:
def lfpallfiles_extract(files):
    if 'lfpdata' in locals():
        del lfpdata
    
    for i, file in enumerate(files):
        
        ### load data
        matdat = sio.loadmat(file, variable_names = variablesinLoadfile, 
                             struct_as_record = False, squeeze_me = True) 

        # fs: sample rate
        fs = matdat['fs'] 
    

        ### dealing lfp data
        
        # lfp (np.ndarray): ntemporal * nchns
        lfp_m1 = matdat['lfpsegs_m1']
        lfp_stn, lfp_gp = matdat['lfpsegs_stn'], matdat['lfpsegs_gp']
        
        

        # combine the data of all the channels
        lfpdata_1file = np.concatenate((lfp_m1, lfp_stn, lfp_gp), axis = 1)
        
        # concatenate to lfpdata for all files
        if 'lfpdata' not in locals():
            lfpdata = lfpdata_1file
        else:
            lfpdata = np.concatenate((lfpdata, lfpdata_1file), axis = 2)
                
                
    
    return lfpdata

In [13]:
files_normal = glob.glob(os.path.join(inputfolder, '*_normal_*'))
files_mild = glob.glob(os.path.join(inputfolder, '*_mild_*'))

In [14]:
print(len(files_normal))
print(len(files_mild))

3
2


In [15]:
lfpwins_normal = lfpallfiles_extract(files_normal)
lfpwins_mild = lfpallfiles_extract(files_mild)

In [16]:
if halfs:
    lfpwins_normal_2s, lfpwins_mild_2s, lfpwins_moderate_2s = lfpwins_normal, lfpwins_mild, lfpwins_moderate
    
    lfpwins_normal = lfpwins_normal_2s[0:250, :,:]
    lfpwins_mild = lfpwins_mild_2s[0:250, :,:]
    lfpwins_moderate = lfpwins_moderate_2s[0:250, :,:]
    
    savefilename = savefilename + '_halfs'

In [17]:
print(lfpwins_normal.shape)
print(lfpwins_mild.shape)

(1000, 110, 172)
(1000, 110, 155)


## Extract chnInf using recchni_used

In [18]:
### load channel information ###
df = pd.read_csv(chnInf_file, header = 0)

In [19]:
# recording chni 
chnInf_recchni = df['recording_chni'].tolist()

chnInf_brainarea = df['brainarea'].tolist()
chnInf_matchni = df['mat_chni']


# coord
x, y = df['simulated_x'].to_numpy(), df['simulated_y'].to_numpy()
chnInf_coord = np.concatenate((np.expand_dims(x, axis = 1), np.expand_dims(y, axis = 1)), axis = 1)

## Calculate ciCOH

### balance mild and normal trials

In [20]:
# select the smaller trial number
ntrials_normal, ntrials_mild = lfpwins_normal.shape[2], lfpwins_mild.shape[2]
#ntrials_moderate = lfpwins_moderate.shape[2]
ntrials = min([ntrials_normal, ntrials_mild])

# balance trials
lfpwins_normal = lfpwins_normal[:,:,0:ntrials]
lfpwins_mild = lfpwins_mild[:,:,0:ntrials]
#lfpwins_moderate = lfpwins_moderate[:,:,0:ntrials]

###  normal ciCOH

In [21]:
lfpwins_allfiles = lfpwins_normal

### calculate ciCOH
ntempo, nchns, nwins = lfpwins_allfiles.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    
    if wini % 10 == 0:
        print("wini = " + str(wini) + "/" + str(nwins))
    
    for chni in range(nchns -1):
        signal1 = lfpwins_allfiles[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins_allfiles[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

wini = 0/155
wini = 10/155
wini = 20/155
wini = 30/155
wini = 40/155
wini = 50/155
wini = 60/155
wini = 70/155
wini = 80/155
wini = 90/155
wini = 100/155
wini = 110/155
wini = 120/155
wini = 130/155
wini = 140/155
wini = 150/155


In [22]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)
ciCOH_normal = ciCOH
del ciCOH, ciCOH_allWins

### mild ciCOH

In [23]:
lfpwins_allfiles = lfpwins_mild

### calculate ciCOH
ntempo, nchns, nwins = lfpwins_allfiles.shape
ciCOH_allWins = np.zeros((nchns, nchns, nwins))
for wini in range(nwins):
    
    if wini%10 ==0:
        print("wini = " + str(wini) + "/" + str(nwins))
        
    for chni in range(nchns -1):
        signal1 = lfpwins_allfiles[:, chni, wini]
        
        for chnj in range(chni+1, nchns):
            signal2 = lfpwins_allfiles[:, chnj, wini]
            
            # ciCOH_allWins assignment
            ciCOH_allWins[chni, chnj, wini] = ciCoherence_overtime(signal1, signal2)
            
            # symmetrical
            ciCOH_allWins[chnj, chni, wini] = ciCOH_allWins[chni, chnj, wini]
            
            del signal2
        del signal1

wini = 0/155
wini = 10/155
wini = 20/155
wini = 30/155
wini = 40/155
wini = 50/155
wini = 60/155
wini = 70/155
wini = 80/155
wini = 90/155
wini = 100/155
wini = 110/155
wini = 120/155
wini = 130/155
wini = 140/155
wini = 150/155


In [24]:
ciCOH = np.mean(ciCOH_allWins, axis = 2)
ciCOH_mild = ciCOH
del ciCOH, ciCOH_allWins

## save ciCOH

In [25]:
ciCOH = dict()
ciCOH['normal'], ciCOH['mild']  = ciCOH_normal, ciCOH_mild, 
#ciCOH['moderate'] = ciCOH_moderate
ciCOH['chnInf_recchni'], ciCOH['chnInf_matchni'] = chnInf_recchni, chnInf_matchni
ciCOH['chnInf_brainarea'], ciCOH['chnInf_coord'] = chnInf_brainarea, chnInf_coord
ciCOH['ntrials'] = ntrials

In [26]:
try:
    import cPickle as pickle
except ImportError:  # python 3.x
    import pickle

with open(os.path.join(savefolder, savefilename + '.pickle'), 'wb') as fp:
    pickle.dump(ciCOH, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
os.path.join(savefolder, savefilename)

'/home/lingling/Insync/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/pipeline/NHP_Jo/1_dataAnaly/FCAnaly/Rest/m1_calcciCOH_rest/ciCOH_rest_freq16_18'